In [2]:
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns


# Load datasets
customers = pd.read_csv('/content/Customers.csv')
products = pd.read_csv('/content/Products.csv')
transactions = pd.read_csv('/content/Transaction.csv')

# Explore datasets
print(customers.info())
print(products.info())
print(transactions.info())

# Convert dates
customers['SignupDate'] = pd.to_datetime(customers['SignupDate'])
transactions['TransactionDate'] = pd.to_datetime(transactions['TransactionDate'])

# Merge datasets
merged_data = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')

# Example insights
# Top-selling products
top_products = merged_data.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False)

# Regional trends
region_sales = merged_data.groupby('Region')['TotalValue'].sum()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    200 non-null    object
 1   CustomerName  200 non-null    object
 2   Region        200 non-null    object
 3   SignupDate    200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ProductID    100 non-null    object 
 1   ProductName  100 non-null    object 
 2   Category     100 non-null    object 
 3   Price        100 non-null    float64
dtypes: float64(1), object(3)
memory usage: 3.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------

<ipython-input-2-4485977e113b>:18: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  transactions['TransactionDate'] = pd.to_datetime(transactions['TransactionDate'])


"sns.barplot(x=region_sales.index, y=region_sales.values)\nplt.title('Total Sales by Region')\nplt.show()"

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import numpy as np

# Feature engineering
customer_features = merged_data.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'TransactionID': 'count'
}).rename(columns={'TransactionID': 'TransactionCount'})

# Normalize features
scaler = StandardScaler()
normalized_features = scaler.fit_transform(customer_features)

# Compute similarity
similarity_matrix = cosine_similarity(normalized_features)

# Recommendations
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features.index, columns=customer_features.index)
def get_top_n_similar(customers, n=3):
    recommendations = {}
    for customer in customers:
        similar_customers = similarity_df[customer].sort_values(ascending=False).iloc[1:n+1]
        recommendations[customer] = list(similar_customers.index)
    return recommendations

# Generate recommendations for first 20 customers
lookalikes = get_top_n_similar(customer_features.index[:20])
lookalikes_df = pd.DataFrame.from_dict(lookalikes, orient='index', columns=[f'Similar_{i+1}' for i in range(3)])
lookalikes_df.to_csv('FirstName_LastName_Lookalike.csv', index_label='CustomerID')

In [4]:
df=pd.read_csv('/content/FirstName_LastName_Lookalike.csv')
print(df)

   CustomerID Similar_1 Similar_2 Similar_3
0       C0001     C0164     C0103     C0069
1       C0002     C0029     C0031     C0077
2       C0003     C0176     C0027     C0010
3       C0004     C0075     C0165     C0113
4       C0005     C0123     C0131     C0058
5       C0006     C0079     C0117     C0196
6       C0007     C0125     C0140     C0092
7       C0008     C0179     C0081     C0084
8       C0009     C0192     C0083     C0177
9       C0010     C0142     C0029     C0027
10      C0011     C0096     C0171     C0064
11      C0012     C0068     C0065     C0102
12      C0013     C0188     C0059     C0022
13      C0014     C0033     C0186     C0095
14      C0015     C0058     C0131     C0005
15      C0016     C0107     C0152     C0183
16      C0017     C0113     C0162     C0124
17      C0018     C0182     C0148     C0200
18      C0019     C0172     C0116     C0157
19      C0020     C0080     C0167     C0063
